# Compute projections
Execute `project_roi.sh` shell script to start Slurm jobs to compute tile projections:

`./project_roi.sh becalia_rabies_barseq/BRYC65.1d/brain/slide_008/section_02/ 1 8 8 7`

# Import requirements

In [ ]:
import iss_preprocess as iss
import numpy as np
from flexiznam.config import PARAMETERS
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd 
from itertools import cycle

# Estimate registration for rounds and channels

In [ ]:
ref_tile = (1,2,7)
data_path = 'becalia_rabies_barseq/BRYC65.1d/brain/slide_008/section_02/'
processed_path = Path(PARAMETERS['data_root']['processed'])

In [ ]:
stack = iss.pipeline.load_processed_tile(data_path, ref_tile)
tforms = iss.reg.register_channels_and_rounds(stack)
save_path = processed_path / data_path / 'tforms.npy'
np.save(save_path, tforms, allow_pickle=True)

# Prepare parameters for OMP

In [ ]:
stack, bad_pixels = iss.pipeline.load_and_register_tile(data_path, ref_tile)
gene_dict, gene_names, norm_shift = iss.pipeline.setup_omp(stack)
save_path = processed_path / data_path / 'gene_dict.npz'
np.savez(save_path, gene_dict=gene_dict, gene_names=gene_names, norm_shift=norm_shift)

# Run OMP
Execute `extract_roi.sh` shell script to start Slurm jobs to run OMP on each tile:

`./extract_roi.sh becalia_rabies_barseq/BRYC65.1d/brain/slide_008/section_02/ 1 8 8`

# Merge OMP output

In [ ]:
shift_right, shift_down, tile_shape = iss.pipeline.register_adjacent_tiles(data_path, ref_coors=ref_tile)
spots = iss.pipeline.merge_roi_spots(data_path, shift_right, shift_down, tile_shape, (9,9))
save_path = Path(PARAMETERS['data_root']['processed']) / data_path / 'gene_spots.pkl' 
spots.to_pickle(save_path)

In [ ]:
plt.style.use("default")

colors = cycle([ 'deepskyblue', 'aquamarine', 'orangered', 'violet', 'forestgreen', 'darkorange'])
markers = cycle('ov^<>spPXD*')
plt.figure(figsize=(18,120))

gene_names = spots['gene'].unique()

for igene, gene in enumerate(gene_names):
    plt.subplot(30,3,igene+1)
    plt.title(gene)
    plt.gca().set_aspect('equal', 'box')

    plt.plot(
        spots[spots['gene'] == gene]['x'],
        spots[spots['gene'] == gene]['y'],        
        '.',
        alpha=0.5,
        markersize=0.5
    )
    plt.gca().invert_yaxis()
    plt.axis('off')

# Generate an overview image and segment cells

In [ ]:
tile_origins, tile_centers = iss.pipeline.calculate_tile_positions(shift_right, shift_down, tile_shape, (9,9))

stitched_stack = iss.pipeline.stitch_tiles(data_path, 'round_01_1', tile_origins, tile_shape)

from cellpose.models import CellposeModel
model = CellposeModel(gpu=False, model_type='cyto', net_avg=True)
masks, flows, styles = model.eval(
    stitched_stack,
    rescale=.55,
    channels=[0,0],
    flow_threshold=0.4,
    tile=True
)
np.save(processed_path / data_path / 'masks_1.npy', masks)